<a href="https://colab.research.google.com/github/ye422/hatedog/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask flask-cors flask_sqlalchemy python-dotenv langchain_community faiss-cpu sentence-transformers transformers torch accelerate einops langchain-openai openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [ ]:
import os
from getpass import getpass

os.environ["HF_TOKEN"] = getpass("Enter HuggingFace Token: ")
os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI Key: ")
os.environ["NGROK_AUTH_TOKEN"] = getpass("Enter ngrok auth token: ")

from huggingface_hub import login
login(os.getenv('HF_TOKEN'))

Enter HuggingFace Token: ··········
Enter OpenAI Key: ··········
Enter ngrok auth token: ··········


In [ ]:
!git clone https://github.com/hatedogs/hatedog.git
%cd hatedog/llm_server

Cloning into 'hatedog'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 29 (delta 5), reused 25 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 61.69 KiB | 928.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/hatedog/llm_server


In [6]:
# ngrok 설치
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz -O ngrok-v3-stable-linux-amd64.tgz

!tar -xvzf ngrok-v3-stable-linux-amd64.tgz

# ngrok authtoken 설정
NGROK_AUTH_TOKEN = os.getenv('NGROK_AUTH_TOKEN')
!./ngrok authtoken $NGROK_AUTH_TOKEN

# ngrok 실행하여 Flask 기본 포트 5000 노출 (백그라운드 실행)
get_ipython().system_raw('./ngrok http 5000 &')

# 공개 URL 가져오기 (ngrok 연결까지 몇 초 대기)
!sleep 8 # 연결 시간을 조금 더 넉넉하게 줌
import requests
import json # json 파싱을 위해 추가

try:
    # ngrok 로컬 API 엔드포인트
    resp = requests.get('http://127.0.0.1:4040/api/tunnels')
    resp.raise_for_status() # HTTP 오류 발생 시 예외 발생
    tunnels_json = resp.json()

    # 터널 정보에서 https URL 찾기
    public_url = None
    for tunnel in tunnels_json.get("tunnels", []):
        if tunnel.get("public_url", "").startswith("https://"):
            public_url = tunnel["public_url"]
            break # https URL을 찾으면 중단

    if public_url:
        print(f"Flask 앱 접근 가능 주소 (ngrok public_url): {public_url}")
        print("이 주소를 Chrome 확장 프로그램의 content.js (API_BASE_URL)와 manifest.json (host_permissions)에 사용하세요.")
    else:
        print("ngrok에서 https 터널을 찾지 못했습니다. ngrok 상태를 확인하세요: http://127.0.0.1:4040")
        print("응답 내용:", json.dumps(tunnels_json, indent=2))


except requests.exceptions.RequestException as e:
    print(f"ngrok API 요청 실패: {e}")
    print("ngrok이 정상적으로 실행되었는지, 포트 포워딩이 올바르게 설정되었는지 확인하세요.")
    print("터미널에서 `./ngrok http 5000` 명령을 직접 실행하여 상태를 볼 수도 있습니다.")
except Exception as e:
    print(f"ngrok 공개 URL 가져오기 중 예상치 못한 오류: {e}")
    print("ngrok 로그(http://127.0.0.1:4040) 또는 Colab 터미널 출력을 확인하세요.")

--2025-05-24 15:59:35--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 13.248.244.96, 35.71.179.82, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|13.248.244.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9395172 (9.0M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.96M  11.5MB/s    in 0.8s    

2025-05-24 15:59:36 (11.5 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9395172/9395172]

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Flask 앱 접근 가능 주소 (ngrok public_url): https://1072-34-16-175-142.ngrok-free.app
이 주소를 Chrome 확장 프로그램의 content.js (API_BASE_URL)와 manifest.json (host_permissions)에 사용하세요.


In [ ]:
!python app.py

2025-05-24 16:00:06.518567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748102406.743508    2295 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748102406.803350    2295 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-24 16:00:07.339728: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:app:Flask 앱: LLM 구성 요소 초기화 시작...
INFO:app:Flask 앱: 사용 장치: cuda
tokenizer_config.json: 100% 61.0/61.0 [00:00<00:0